## Making Lane Detector

In [2]:
import cv2 as cv
import numpy as np

In [30]:
class PrepareImage():
    '''ATTRIBUTES:
    gauss_size : kernel size for the gaussian blur 
                type-> tuple of size 2 with odd and equal 
                       entries > 1
    gauss_deviation : x and y axis standard deviations for 
               gaussian blur 
               type -> list-like of size = 2
    auto_canny : If auto canny is True use median of blurred 
                image to calculate thresholds 
                type-> boolean
    canny_low : the lower threshold of the canny filter 
                type -> int 
    canny_high : the higher threshold of the canny filter 
                type -> int 
    segment_x : the width of segment peak( the triangular 
               segment head). Given as the fraction of the width 
               of the image
               type -> float in (0,1) 0 and 1 exclusive
    segment_y : the height segment peak
                Given as the fraction of the height from the 
                top 
                type -> float in (0,1) 0 and 1 exclusive
                
    METHODS:
    ...
    '''
    def __init__(self,
                gauss_size = None,
                gauss_deviation = None,
                auto_canny = False,
                canny_low = 50,
                canny_high = 175,
                segment_x = 0.5,
                segment_y = 0.5):
        
        # setting gaussian kernel parameters.
        if(gauss_size is not None):
            if(len(gauss_size) != 2):
                raise Exception("Wrong size for the Gaussian Kernel")
            elif(type(gauss_size) is not tuple):
                raise Exception("Kernel type should be a tuple")
            elif(gauss_size[0]%2 == 0 or gauss_size[1]%2 == 0):
                raise Exception("Even entries found in Gaussian Kernel")    
        self.gauss_kernel = gauss_size
            
        if(gauss_deviation is not None):
            if(len(gauss_deviation)!=2):
                raise Exception("Wrong length of gauss deviation")
            else:
                self.gauss_deviation = gauss_deviation
            
        if(type(auto_canny) is not bool):
            raise TypeError("Incorrect Type mentioned for auto canny")
            
        # setting canny parameters
        if(auto_canny is False):
            self.auto_canny = False
            if(type(canny_high) is int and type(canny_low) is int):
                self.canny_low = canny_low 
                self.canny_high = canny_high 
            else:
                raise TypeError("Incorrect type specified for canny thresholds")
        else:
            self.auto_canny = True
            
        # setting segment parameters
        if segment_x >=1 or segment_x<=0:
            raise Exception("Fraction specified is out of range (0,1)")
        else:
            self.segment_x = segment_x
        if segment_y >=1 or segment_y<=0:
            raise Exception("Fraction specified is out of range (0,1)")
        else:
            self.segment_y = segment_y 
    def get_binary_image(self,frame):
        can = self.do_canny(frame)
        seg = self.segment_image(can)
        buffers = self.draw_probable_lane(seg)
        return (seg,buffers)
    def do_canny(self,frame):
        '''PARAMETERS: the frame of the image on which we want to apply the 
                      canny filter 
          RETURNS : a canny filtered frame '''
        # gray the image 
        gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY) 
        # apply blur 
        if(self.gauss_kernel is None):
            self.gauss_kernel = (9,9) # using a default kernel size 
        if(self.gauss_deviation is None):
            self.gauss_deviation = [3,3]
        
        blur = cv.GaussianBlur(gray, self.gauss_kernel, self.gauss_deviation[0], self.gauss_deviation[1])
        
        #apply canny filter 
        if self.auto_canny is False:
            canny = cv.Canny(blur,self.canny_low,self.canny_high)
        else:
            # Auto canny trumps specified parameters 
            v = np.median(blur)
            sigma = 0.33
            lower = int(max(0, (1.0 - sigma) * v))
            upper = int(min(255, (1.0 + sigma) * v))
            canny = cv.Canny(blur,lower,upper)
        
        return canny 
    
    def segment_image(self,frame):
        '''PARAMETERS: the frame of the image on which we want to apply the 
                      segementation filter 
        RETURNS : a segmented canny filtered frame '''
        height = frame.shape[0]
        width = frame.shape[1]
        shift = int(0.08 * width)
        points = np.array([
            [(0,height),(width,height),(int(width*self.segment_x)+shift,int(height*self.segment_y)),
             (int(width*self.segment_x)-shift,int(height*self.segment_y))]
        ])
        # create an image with zero intensity with same dimensions as frame.
        mask = np.zeros_like(frame)
        
        cv.fillPoly(mask,points,255) # filling the frame's triangle with white pixels
        # do a bitwise and on the canny filtered black and white image and the 
        # segment you just created to get a triangular area for lane detection 
        segment = cv.bitwise_and(frame, mask)
        return segment 
    
    def draw_probable_lane(self,frame):
        '''PARAMETERS : the image frame we got from the segmentation filter 
        RETURNS : a buffer frame so that we can provide some kind of output 
        if no lane is currently detected. '''
        img = frame
        imgright = frame 
        imgleft = frame
        height = frame.shape[0]
        width = frame.shape[1]
        shift_above = int(0.08 * width)
        shift_below = int(0.25 * width)
        # left lane points 
        xy_1l = (shift_below,height) # bottom left
        xy_2l = (int(self.segment_x * width) - shift_above , int(self.segment_y * height) ) # center left 
        # right lane points 
        xy_1r = (width - shift_below,height) #bottom right
        xy_2r = (int(self.segment_x * width) + shift_above , int(self.segment_y * height) ) # center right
        # draw lines
        cv.line(img,xy_1l,xy_2l,(200,200,200),5)
        cv.line(img,xy_1r,xy_2r,(200,200,200),5)
        cv.line(imgleft,xy_1l,xy_2l,(200,200,200),5)
        cv.imshow("image",imgleft)
        cv.waitKey(3000)
        cv.destroyAllWindows()
        cv.line(imgright,xy_1r,xy_2r,(200,200,200),5)
        return (imgleft,imgright,img)
    
    ## to do more....

In [11]:
def rescale_frame(self,frame,percent=75):
        width = int(frame.shape[1] * percent / 100)
        height = int(frame.shape[0] * percent / 100)
        dim = (width,height)
        return cv.resize(frame,dim,interpolation = cv.INTER_AREA)

In [31]:
prep = PrepareImage((11,11),(3,3),auto_canny=True, segment_y = 0.5)

In [33]:
#working fine...
cap = cv.VideoCapture(r"E:\InnerveHackathon\pathvalild.mp4")
while True:
    got,img = cap.read()
    if got:
        images = prep.get_binary_image(img)
        cv.line(images[1][0],(images[1][0].shape[1]//2,0),(images[1][0].shape[1]//2,images[1][0].shape[0]),(200,200,0),5)
        cv.imshow("Final",images[1][1])
        if cv.waitKey(15) & 0xFF ==ord('q'):
            break
cv.destroyAllWindows()

## So, we will show atleast a basic navigation information if no lane is actually detected
- Made the buffer lanes for the output for lanes when there actually isn't any lane detected throgh Hough Transformations
